# Google Analytics Customer Revenue Prediction

* Data Exploration
* 2018.09.17 ~

## 1. Import Required Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
%matplotlib inline

## 2. Data Loading

In [2]:
train=pd.read_csv('../data/final_train.csv')
test=pd.read_csv('../data/final_test.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 3. Feature Exploration

### Data Fields
* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you *  should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

In [3]:
train.head()

,Unnamed: 0,fullVisitorId,visitNumber,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.campaign,...,channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Display,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet
0,0,1131660440785968503,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,...,0,0,0,1,0,0,0,1,0,0
1,1,377306020877927890,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,...,0,0,0,1,0,0,0,1,0,0
2,2,3895546263509774583,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,...,0,0,0,1,0,0,0,1,0,0
3,3,4763447161404445595,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,...,0,0,0,1,0,0,0,1,0,0
4,4,27294437909732085,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0,...,0,0,0,1,0,0,0,0,1,0


In [4]:
test.head()

,Unnamed: 0,fullVisitorId,visitNumber,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.campaign,trafficSource.keyword,...,channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Display,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet
0,0,6167871330617112363,2.0,0.0,4.0,0.0,4.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,1,0643697640977915618,1.0,0.0,5.0,1.0,5.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,2,6059383810968229466,1.0,0.0,7.0,1.0,7.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,3,2376720078563423631,1.0,0.0,8.0,1.0,4.0,1.0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,4,2314544520795440038,1.0,0.0,9.0,1.0,4.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [5]:
train=train.iloc[:, 1:]

In [6]:
test=test.iloc[:, 1:]

In [7]:
train.head()

,fullVisitorId,visitNumber,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.campaign,trafficSource.keyword,...,channelGrouping_Affiliates,channelGrouping_Direct,channelGrouping_Display,channelGrouping_Organic Search,channelGrouping_Paid Search,channelGrouping_Referral,channelGrouping_Social,device.deviceCategory_desktop,device.deviceCategory_mobile,device.deviceCategory_tablet
0,1131660440785968503,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,377306020877927890,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,3895546263509774583,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,4763447161404445595,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0,1,...,0,0,0,1,0,0,0,1,0,0
4,27294437909732085,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [17]:
test = test.drop("trafficSource.adwordsClickInfo.adNetworkType_Content", axis=1)

## 4. Train

In [18]:
train.columns

Index(['fullVisitorId', 'visitNumber', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
       'totals.visits', 'trafficSource.campaign', 'trafficSource.keyword',
       'year', 'month', 'day', 'week',
       'trafficSource.adwordsClickInfo.adNetworkType_Google Search',
       'trafficSource.adwordsClickInfo.adNetworkType_Search partners',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'device.deviceCategory_desktop', 'device.deviceCategory_mobile',
       'device.deviceCategory_tablet'],
      dtype='object')

In [19]:
col = test.columns

In [20]:
col

Index(['fullVisitorId', 'visitNumber', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.visits',
       'trafficSource.campaign', 'trafficSource.keyword', 'year', 'month',
       'day', 'week',
       'trafficSource.adwordsClickInfo.adNetworkType_Google Search',
       'trafficSource.adwordsClickInfo.adNetworkType_Search partners',
       'channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'device.deviceCategory_desktop', 'device.deviceCategory_mobile',
       'device.deviceCategory_tablet'],
      dtype='object')

In [21]:
x_train = train[col]

In [22]:
y_train = train['totals.transactionRevenue']

## 4. Using RandomForest

In [29]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=35)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=35, verbose=0, warm_start=False)

## 5. Validation

In [30]:
from sklearn.model_selection import cross_val_predict
y_predict = cross_val_predict(model, x_train, y_train, cv=20)
print(y_predict.shape)
y_predict

(903653,)


array([      0.,       0.,       0., ...,       0., 2718000., 8791000.])

## 6. Evaluate

In [32]:
import math
score = math.sqrt(((y_predict - y_train)**2).mean())
score

56112334.134341

## 7. Training

In [33]:
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=35, verbose=0, warm_start=False)

In [67]:
import numpy as np
prediction = model.predict(test)
prediction = np.log1p(prediction)

In [68]:
prediction = pd.DataFrame(prediction, columns=['totals.transactionRevenue'])

In [69]:
prediction.shape

(804684, 1)

In [70]:
test.shape

(804684, 26)

In [71]:
test_cat = test['fullVisitorId']

In [72]:
test_cat = test_cat.astype(str)

In [73]:
test_cat = pd.DataFrame(test_cat, columns=["fullVisitorId"])

In [74]:
test_cat.shape

(804684, 1)

In [75]:
prediction.shape

(804684, 1)

In [76]:
prediction = pd.concat([test_cat, prediction], axis=1)

In [77]:
prediction

,fullVisitorId,totals.transactionRevenue
0,6167871330617112363,0.000000
1,0643697640977915618,0.000000
2,6059383810968229466,0.000000
3,2376720078563423631,0.000000
4,2314544520795440038,0.000000
5,4133039884103392367,0.000000
6,4320478850207397557,14.456839
7,5876438247590157131,14.508158
8,0514591268737702944,0.000000
9,6430567031531677212,0.000000
